# Group 20 :
## CS23MTECH11002 - Anirudh Joshi
## CS23MTECH11008 - V Harikrishnan
## AI23MTECH11004 - Koushik Maji


## Run sequentially (GPU required) if ample RAM is present, else restart session after training 1 model and run common cells along with the model.

# Common cells

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install transformers -U

In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d649fe97bc3c04fdd2c8c6458b46aab276b339e34a130b3385948940c78a65c7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
!pip install nltk

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#### Importing the required libraries

In [1]:
import warnings
import torch.nn.functional as F
from datasets import DatasetDict
import pickle
from transformers import ViTModel, GPT2LMHeadModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor, GPT2Config, default_data_collator
import torch
import torch.nn as nn
from transformers.modeling_outputs import BaseModelOutputWithPooling
import evaluate
import numpy as np
import nltk
warnings.filterwarnings('ignore')

In [2]:

# Replace with the path to the pickle file (dataset)
with open('/content/drive/MyDrive/processed_dataset_f.pkl', 'rb') as f:
    processed_dataset = pickle.load(f)

In [3]:
subset_processed_datadict = DatasetDict({
    'train': processed_dataset['train'].select(range(200))
})

eval_dataset = processed_dataset['train'].select(range(300, 400))

subset_processed_datadict['validation'] = eval_dataset


#### Run this cell if facing RAM issues (not required)


In [4]:
del processed_dataset

In [5]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    output_dir="./image-captioning-output",
)

In [6]:
metric = evaluate.load("rouge")

In [7]:
ignore_pad_token_for_loss = True
nltk.download('punkt')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Model without multi modal attention

In [ ]:

class Vit_encoder(torch.nn.Module):
    def __init__(self):
        super(Vit_encoder, self).__init__()

        self.encoder = ViTModel.from_pretrained('google/vit-base-patch16-224')
        self.config = self.encoder.config
        self.main_input_name = self.encoder.main_input_name

    def get_output_embeddings(self):
      return None

    def forward(self, pixel_values, decoder_input_ids=None, labels=None, **kwargs):

        encoder_outputs = self.encoder(pixel_values=pixel_values)
        del pixel_values

        return encoder_outputs


In [ ]:
image_encoder_model = "google/vit-base-patch16-224-in21k"
text_decode_model = "gpt2"

config = GPT2Config.from_pretrained('gpt2')
config.add_cross_attention = True

feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

gpt2_decoder = GPT2LMHeadModel.from_pretrained('gpt2', config=config)
tokenizer.pad_token = tokenizer.eos_token
torch.manual_seed(50)
torch.cuda.manual_seed_all(50)
vit_model = Vit_encoder()
gpt2_decoder.pad_token_id = tokenizer.pad_token_id
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
gpt2_decoder.resize_token_embeddings(len(tokenizer))
norm_model = VisionEncoderDecoderModel(encoder=vit_model, decoder=gpt2_decoder)
norm_model.config.eos_token_id = tokenizer.eos_token_id
norm_model.config.decoder_start_token_id = tokenizer.bos_token_id
norm_model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.c_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.ln_cross_attn.bias', 'h.0.ln_cross_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.ln_cross_attn.bias', 'h.1.ln_cross_attn.weight', 'h.10.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.10.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.weight', 'h.10.crossattention.q_attn.bias', 'h.10.crossattention.q_attn.weight', 'h.10.ln_cross_attn.bias', 'h.10.ln_cross_attn.weight', 'h.11.crossattention.c_attn.bias', 'h.11.crossattention.c_attn.weight', 'h.11.crossat

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Seq2SeqTrainer(
    model=norm_model,
    args=training_args,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    train_dataset=subset_processed_datadict['train'],
    eval_dataset=subset_processed_datadict['validation'],
    data_collator=default_data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.675264,7.566000,0.095200,6.811300,6.771400,19.860000
2,No log,0.672297,7.602800,0.866000,7.316600,7.308000,19.740000
3,No log,0.640696,8.787700,0.639100,7.932900,8.010500,19.260000


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

TrainOutput(global_step=150, training_loss=1.341080322265625, metrics={'train_runtime': 154.3613, 'train_samples_per_second': 3.887, 'train_steps_per_second': 0.972, 'total_flos': 1.082783346720768e+17, 'train_loss': 1.341080322265625, 'epoch': 3.0})

In [ ]:
del trainer

# Model with cross attention

In [8]:
subset_processed_datadict['train'] = subset_processed_datadict['train'].rename_column('pixel_values', 'pixels')
subset_processed_datadict['validation'] = subset_processed_datadict['validation'].rename_column('pixel_values', 'pixels')

In [9]:
def package_inputs(example):
    input_ids = torch.tensor(example['article_embed'])
    pixels = torch.tensor(example['pixels'])
    pad_height = 224 - (512 % 224)
    pad_width = 224 - (768 % 224)
    padded_embed = torch.nn.functional.pad(input_ids, (0, pad_width, 0, pad_height))

    padded_embed_reshaped = padded_embed.view(-1, 224, 224)
    result = torch.cat((pixels, padded_embed_reshaped), dim=0)
    example['pixel_values'] = result.tolist()
    del input_ids
    del pixels
    del padded_embed
    del example['pixels']
    del example['article_embed']
    return example


In [10]:
subset_processed_datadict['train'] = subset_processed_datadict['train'].map(package_inputs)
subset_processed_datadict['validation'] = subset_processed_datadict['validation'].map(package_inputs)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [11]:

class Cross_Attention(torch.nn.Module):
    def __init__(self):
        super(Cross_Attention, self).__init__()

        self.encoder = ViTModel.from_pretrained('google/vit-base-patch16-224')
        self.encoder.config.pad_token_id = self.encoder.config.eos_token_id
        self.config = self.encoder.config

        self.main_input_name = self.encoder.main_input_name
        self.multihead_attn = nn.MultiheadAttention(embed_dim=768, num_heads=8)

    def get_output_embeddings(self):
      return None
    def forward(self, pixel_values, input_ids = None, decoder_input_ids=None, labels=None, flag = False, **kwargs):

        values = pixel_values[:, :3 ,:, :]
        temp = pixel_values[:, 3:, :, :]
        pad_height = 224 - (512 % 224)
        pad_width = 224 - (768 % 224)
        padded_article_embed  = temp.view(pixel_values.size(0), 512 + 224 - (512 % 224), 768 + 224 - (768 % 224))

        article_embed  = padded_article_embed[:, :512, :768]

        encoder_outputs = self.encoder(pixel_values=values)
        queries = encoder_outputs.last_hidden_state.transpose(0, 1)
        keys = article_embed.transpose(0, 1)
        values = article_embed.transpose(0, 1)

        attn_output, _ = self.multihead_attn(queries, keys, values)
        attn_output = attn_output.transpose(0, 1).contiguous()

        new_encoder_outputs = BaseModelOutputWithPooling(
            last_hidden_state=attn_output,
            pooler_output=encoder_outputs.pooler_output,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
        )

        del pixel_values
        del encoder_outputs
        del queries
        del keys
        del values
        del attn_output
        return new_encoder_outputs


In [12]:
image_encoder_model = "google/vit-base-patch16-224-in21k"
text_decode_model = "gpt2"

config = GPT2Config.from_pretrained('gpt2')
config.add_cross_attention = True

feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

gpt2_decoder = GPT2LMHeadModel.from_pretrained('gpt2', config=config)
tokenizer.pad_token = tokenizer.eos_token
torch.manual_seed(46)
torch.cuda.manual_seed_all(57)
vit_with_crossattention = Cross_Attention()
gpt2_decoder.pad_token_id = tokenizer.pad_token_id
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
gpt2_decoder.resize_token_embeddings(len(tokenizer))
final_model = VisionEncoderDecoderModel(encoder=vit_with_crossattention, decoder=gpt2_decoder)
final_model.config.eos_token_id = tokenizer.eos_token_id
final_model.config.decoder_start_token_id = tokenizer.bos_token_id
final_model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.c_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.ln_cross_attn.bias', 'h.0.ln_cross_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.ln_cross_attn.bias', 'h.1.ln_cross_attn.weight', 'h.10.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.10.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.weight', 'h.10.crossattention.q_attn.bias', 'h.10.crossattention.q_attn.weight', 'h.10.ln_cross_attn.bias', 'h.10.ln_cross_attn.weight', 'h.11.crossattention.c_attn.bias', 'h.11.crossattention.c_attn.weight', 'h.11.crossat

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer = Seq2SeqTrainer(
    model=final_model,
    args=training_args,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    train_dataset=subset_processed_datadict['train'],
    eval_dataset=subset_processed_datadict['validation'],
    data_collator=default_data_collator,
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.776521,7.873100,0.047600,7.297000,7.342300,19.940000
2,No log,0.738688,9.918300,0.152400,8.511700,8.605800,20.000000
3,No log,0.730840,9.816300,0.047600,8.486700,8.599800,20.000000


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

TrainOutput(global_step=150, training_loss=1.2642074584960938, metrics={'train_runtime': 564.6582, 'train_samples_per_second': 1.063, 'train_steps_per_second': 0.266, 'total_flos': 5.47792518905856e+17, 'train_loss': 1.2642074584960938, 'epoch': 3.0})